In [1]:
import tensorflow as tf
import cv2
import sys
import pong_fun as game
import random
import time 
import numpy as np
from collections import deque

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
ACTIONS = 6 # number of valid actions
GAMMA = 0.99 # decay rate of past observations
OBSERVE = 500. # timesteps to observe before training
EXPLORE = 500. # frames over which to anneal epsilon
FINAL_EPSILON =  0.05 # final value of epsilon
INITIAL_EPSILON = 1 # starting value of epsilon
REPLAY_MEMORY = 50000 # number of previous transitions to remember
BATCH = 32 # size of minibatch

In [3]:
np.random.seed(0)
tf.set_random_seed(0)
# Deep Q Network off-policy
class DeepQNetwork:
    def __init__(
            self,
            ACTIONS = 6,
            GAMMA = 0.99,
            OBSERVE = 500., # timesteps to observe before training
            EXPLORE = 500., # frames over which to anneal epsilon
            FINAL_EPSILON =  0.05, # final value of epsilon
            INITIAL_EPSILON = 1, # starting value of epsilon
            REPLAY_MEMORY = 50000,# number of previous transitions to remember
            BATCH = 32, # size of minibatch
            input_size = [84,84,4],
            learning_rate = 1e-6,
            replace_target_iter=50,
    ):
        tf.reset_default_graph()
        self.n_actions = ACTIONS
        self.gamma = GAMMA
        self.observe = OBSERVE  # timesteps to observe before training
        self.explore = EXPLORE # frames over which to anneal epsilon
        self.final_epsilon = FINAL_EPSILON  # final value of epsilon
        self.initial_epsilon = INITIAL_EPSILON  # starting value of epsilon
        self.memory_size = REPLAY_MEMORY # number of previous transitions to remember
        self.batch_size = BATCH # size of minibatch
        self.input_size =input_size 
        self.lr = learning_rate
        self.replace_target_iter = replace_target_iter

        # total learning step
        self.time_step = 0       
        self.epsilon = INITIAL_EPSILON
        #initialize zero memory [s, a, r, s_]
        self.memory = deque()

        # consist of [target_net, evaluate_net]
        self._build_net()

        t_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='target_net')
        e_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='eval_net')

        with tf.variable_scope('soft_replacement'):
            self.target_replace_op = [tf.assign(t, e) for t, e in zip(t_params, e_params)]

        self.sess = tf.Session()

        self.sess.run(tf.global_variables_initializer())
        print("---------- Class for the Agent is Ready ------------")
        
    # Building the Deep Q-Network
    def _build_net(self):
        # ------------------ all inputs ------------------------
        self.s = tf.placeholder(tf.float32,[None,self.input_size[0],self.input_size[1],self.input_size[2]],name='s') 
        self.s_next = tf.placeholder(tf.float32,[None,self.input_size[0],self.input_size[1],self.input_size[2]],name='s_')
        self.r = tf.placeholder(tf.float32, [None, ], name='r')  # input Reward
        self.a = tf.placeholder(tf.float32, [None, ACTIONS], name='a')
        self.y = tf.placeholder(tf.float32, [None])

        # Gausian weight initializer
        w_initializer, b_initializer = tf.random_normal_initializer(0., 0.01), tf.constant_initializer(0.01)
        
        # ------------------ build evaluate_net ------------------
        with tf.variable_scope('eval_net'):
            # Convolutional Layer 1
            with tf.variable_scope('conv1'): # hidden layer -1
                # 5x5 conv, 1 input, 32 outputs
                w1 = tf.get_variable('w1',[8, 8, self.input_size[2], 32],initializer=w_initializer)
                b1 = tf.get_variable('b1',[32],initializer=b_initializer)
                # Convolution Layer
                l1 = tf.nn.conv2d(self.s, w1, strides=[1, 4, 4, 1], padding="VALID")
                conv1 = tf.nn.relu(tf.nn.bias_add(l1, b1) )
                #print(conv1.get_shape())
            with tf.variable_scope('conv2'): # hidden layer -2
                # 5x5 conv, 1 input, 32 outputs
                w2 = tf.get_variable('w2',[4, 4, 32, 64],initializer=w_initializer)
                b2 = tf.get_variable('b2',[64],initializer=b_initializer)
                # Convolution Layer
                l2 = tf.nn.conv2d(conv1, w2, strides=[1, 2, 2, 1], padding="VALID")
                conv2 = tf.nn.relu(tf.nn.bias_add(l2, b2))
                #print(conv2.get_shape())
            with tf.variable_scope('conv3'): # hidden layer -3
                # 5x5 conv, 1 input, 32 outputs
                w3 = tf.get_variable('w3',[3, 3, 64, 64],initializer=w_initializer)
                b3 = tf.get_variable('b3',[64],initializer=b_initializer)
                # Convolution Layer
                l3 = tf.nn.conv2d(conv2, w3, strides=[1, 1, 1, 1], padding="VALID")
                conv3 = tf.nn.relu(tf.nn.bias_add(l3, b3) )
                #print(conv3.get_shape())
                # Reshape conv3 output to fit fully connected layer input
                fc1_input = tf.reshape(conv3, [-1, 7*7*64])
                #print(fc1_input.get_shape())
            with tf.variable_scope('fully_connected_layer'):
                # Fully connected layer
                w4 = tf.get_variable('w4',[7*7*64,512],initializer=w_initializer)
                b4 = tf.get_variable('b4',[1, 512],initializer=b_initializer)
                self.fc_out = tf.nn.relu(tf.matmul(fc1_input, w4) + b4)
                #print(self.fc_out.get_shape())
            with tf.variable_scope("output_layer___1"):
                w5 = tf.get_variable('w5',[512, self.n_actions],initializer=w_initializer)
                b5 = tf.get_variable('b5',[1, self.n_actions],initializer=b_initializer)
                #print(w5.get_shape(),"W5shape")
                #print(b5.get_shape(),"W5shape")
                self.q_eval = tf.matmul(self.fc_out, w5) + b5
                #print(self.q_eval)
                print("Building of the Evaluation network is Done _Dola_Ram")
                
        # ------------------ build target_net ------------------
        with tf.variable_scope('target_net'):
            # Convolutional Layer 1
            with tf.variable_scope('conv1'): # hidden layer -1
                # 5x5 conv, 1 input, 32 outputs
                w1 = tf.get_variable('w1',[8, 8, self.input_size[2], 32],initializer=w_initializer)
                b1 = tf.get_variable('b1',[32],initializer=b_initializer)
                # Convolution Layer
                l1 = tf.nn.conv2d(self.s_next, w1, strides=[1, 4, 4, 1], padding="VALID")
                conv1 = tf.nn.relu(tf.nn.bias_add(l1, b1) )
                #print(conv1.get_shape())
            with tf.variable_scope('conv2'): # hidden layer -2
                # 5x5 conv, 1 input, 32 outputs
                w2 = tf.get_variable('w2',[4, 4, 32, 64],initializer=w_initializer)
                b2 = tf.get_variable('b2',[64],initializer=b_initializer)
                # Convolution Layer
                l2 = tf.nn.conv2d(conv1, w2, strides=[1, 2, 2, 1], padding="VALID")
                conv2 = tf.nn.relu(tf.nn.bias_add(l2, b2))
                #print(conv2.get_shape())
            with tf.variable_scope('conv3'): # hidden layer -3
                # 5x5 conv, 1 input, 32 outputs
                w3 = tf.get_variable('w3',[3, 3, 64, 64],initializer=w_initializer)
                b3 = tf.get_variable('b3',[64],initializer=b_initializer)
                # Convolution Layer
                l3 = tf.nn.conv2d(conv2, w3, strides=[1, 1, 1, 1], padding="VALID")
                conv3 = tf.nn.relu(tf.nn.bias_add(l3, b3) )
                #print(conv3.get_shape())
                # Reshape conv3 output to fit fully connected layer input
                fc1_input = tf.reshape(conv3, [-1, 7*7*64])
                #print(fc1_input.get_shape())
            with tf.variable_scope('fully_connected_layer'):
                # Fully connected layer
                w4 = tf.get_variable('w4',[7*7*64,512],initializer=w_initializer)
                b4 = tf.get_variable('b4',[1, 512],initializer=b_initializer)
                self.fc_out = tf.nn.relu(tf.matmul(fc1_input, w4) + b4)
                #print(self.fc_out.get_shape())
            with tf.variable_scope("output_layer___1"):
                w5 = tf.get_variable('w5',[512, self.n_actions],initializer=w_initializer)
                b5 = tf.get_variable('b5',[1, self.n_actions],initializer=b_initializer)
                #print(w5.get_shape(),"W5shape")
                #print(b5.get_shape(),"W5shape")
                self.q_next = tf.matmul(self.fc_out, w5) + b5
                print("Building of the Target network is Done _Dola_Ram")
                
        # ---------- for loss and chossing actions ---------------
        with tf.variable_scope('q_eval'):
            # current value of the Q 
            self.q_eval_wrt_a = tf.reduce_sum(tf.multiply(self.q_eval, self.a), reduction_indices = 1)
            
        with tf.variable_scope('loss'):
            #  Mean squared loss mean((q_target-q_eval_wrt_a)^2)
            self.cost = tf.reduce_mean(tf.square(self.y - self.q_eval_wrt_a))
 
        with tf.variable_scope('train'):
            self.train_step = tf.train.AdamOptimizer(self.lr).minimize(self.cost)
            
    def obs(self,a, observation):
        actions_value = self.sess.run(self.a_indices, feed_dict={self.a: a,self.s: observation})
        return actions_value
    
    
    #--------- Choosing action based for the state(observation)----------------------------
    def choose_action(self, s_t):
        self.time_step += 1
         # choose an action epsilon greedily
        q_eval_t = self.sess.run(self.q_eval, feed_dict={self.s: s_t})
        # readout_t = readout.eval(feed_dict = {s : [s_t]})[0]
        a_t = np.zeros([self.n_actions])
        if random.random() <= self.epsilon or self.time_step <= self.observe:
            action_index = random.randrange(self.n_actions)
            a_t[action_index] = 1
        else:
            action_index = np.argmax(q_eval_t)
            a_t[action_index] = 1

        # scale down epsilon
        if self.epsilon > self.final_epsilon and self.time_step > self.observe :
            self.epsilon -= (self.initial_epsilon - self.final_epsilon) / self.explore

        return a_t, self.epsilon
    
    # ----------- learn the optimal polocy 
    def learn(self):
        # sample batch memory from all memory
        # only train if done observing
        if (self.time_step > self.observe) :
            # check to replace target parameters
            if self.time_step % self.replace_target_iter == 0:
                self.sess.run(self.target_replace_op)
                # print('\n target_params_replaced \n')
            # sample a minibatch to train on
            minibatch = random.sample(self.memory, self.batch_size)
            # get the batch variables
            s_j_batch = [d[0] for d in minibatch]
            a_batch = [d[1] for d in minibatch]
            r_batch = [d[2] for d in minibatch]
            s_j1_batch = [d[3] for d in minibatch]
            y_batch = []
            #readout_j1_batch = readout.eval(feed_dict = {s : s_j1_batch})
            readout_j1_batch = self.sess.run(self.q_next, feed_dict = {self.s_next : s_j1_batch})
            for i in range(0, len(minibatch)):
                # if terminal only equals reward
                if minibatch[i][4]:
                    y_batch.append(r_batch[i])
                else:
                    y_batch.append(r_batch[i] + self.gamma * np.max(readout_j1_batch[i]))
            # perform gradient step
            '''train_step.run(feed_dict = {
                y : y_batch,
                a : a_batch,
                s : s_j_batch})'''
            _,q_eval = self.sess.run([self.train_step,self.q_eval], feed_dict = {
                self.y : y_batch,
                self.a : a_batch,
                self.s : s_j_batch})
        return q_eval
    
    def store_transition(self, s_t, a_t, r_t, s_t1, terminal):
        # store the transition in replay memory
        self.memory.append((s_t, a_t, r_t, s_t1, terminal))
        if len(self.memory) > REPLAY_MEMORY:
            self.memory.popleft()
            

In [4]:
RL = DeepQNetwork()
# open up a game state to communicate with emulator
game_state = game.GameState()
# get the first state by doing nothing and preprocess the image to 80x80x4
do_nothing = np.zeros(ACTIONS)
do_nothing[0] = 1
x_t, r_0, terminal, bar1_score, bar2_score = game_state.frame_step(do_nothing)
x_t = cv2.cvtColor(cv2.resize(x_t, (84, 84)), cv2.COLOR_BGR2GRAY)
ret, x_t = cv2.threshold(x_t,1,255,cv2.THRESH_BINARY)
s_t = np.stack((x_t, x_t, x_t, x_t), axis = 2)
t = 0
tick = time.time()
while True:
    # choose an action epsilon greedily
    a_t, epsilon = RL.choose_action(s_t.reshape(1,84, 84, 4))
    # run the selected action and observe next state and reward
    x_t1_col, r_t, terminal, bar1_score, bar2_score = game_state.frame_step(a_t)
    x_t1 = cv2.cvtColor(cv2.resize(x_t1_col, (84, 84)), cv2.COLOR_BGR2GRAY)
    ret, x_t1 = cv2.threshold(x_t1,1,255,cv2.THRESH_BINARY)
    x_t1 = np.reshape(x_t1, (84, 84, 1))
    s_t1 = np.append(x_t1, s_t[:,:,0:3], axis = 2)
    
    RL.store_transition(s_t, a_t, r_t, s_t1, terminal)
    
    # only train if done observing
    if (t > OBSERVE) :
        q_eval = RL.learn()
            
    # update the old values
    s_t = s_t1
    t += 1
    if r_t!= 0:
        print ("TIMESTEP", t, "/ EPSILON", epsilon, "/ bar1_score", bar1_score, "/ bar2_score",bar2_score)
               #, "/ REWARD", r_t, "/ Q_MAX %e" % np.max(readout_t))
    if(bar1_score - bar2_score > 18): 
        print("Game_Ends_in Time:",int(time.time() - tick))
        break;

Building of the Evaluation network is Done _Dola_Ram
Building of the Target network is Done _Dola_Ram
---------- Class for the Agent is Ready ------------
TIMESTEP 129 / EPSILON 1 / bar1_score 0 / bar2_score 1
TIMESTEP 175 / EPSILON 1 / bar1_score 0 / bar2_score 2
TIMESTEP 221 / EPSILON 1 / bar1_score 0 / bar2_score 3
TIMESTEP 267 / EPSILON 1 / bar1_score 0 / bar2_score 4
TIMESTEP 313 / EPSILON 1 / bar1_score 0 / bar2_score 5
TIMESTEP 359 / EPSILON 1 / bar1_score 0 / bar2_score 6
TIMESTEP 405 / EPSILON 1 / bar1_score 0 / bar2_score 7
TIMESTEP 451 / EPSILON 1 / bar1_score 0 / bar2_score 8
TIMESTEP 497 / EPSILON 1 / bar1_score 0 / bar2_score 9
TIMESTEP 543 / EPSILON 0.9182999999999995 / bar1_score 0 / bar2_score 10
TIMESTEP 589 / EPSILON 0.8308999999999989 / bar1_score 0 / bar2_score 11
TIMESTEP 635 / EPSILON 0.7434999999999983 / bar1_score 0 / bar2_score 12
TIMESTEP 681 / EPSILON 0.6560999999999977 / bar1_score 0 / bar2_score 13
TIMESTEP 727 / EPSILON 0.5686999999999971 / bar1_score 0 /

TIMESTEP 6736 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 6
TIMESTEP 6782 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 7
TIMESTEP 6828 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 8
TIMESTEP 6874 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 9
TIMESTEP 6920 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 10
TIMESTEP 6966 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 11
TIMESTEP 7012 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 12
TIMESTEP 7058 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 13
TIMESTEP 7104 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 14
TIMESTEP 7150 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 15
TIMESTEP 7196 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 16
TIMESTEP 7242 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 17
TIMESTEP 7288 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 18
TIMESTEP 7334 / EPSILON 0.049

TIMESTEP 13296 / EPSILON 0.04999999999999416 / bar1_score 1 / bar2_score 9
TIMESTEP 13342 / EPSILON 0.04999999999999416 / bar1_score 1 / bar2_score 10
TIMESTEP 13388 / EPSILON 0.04999999999999416 / bar1_score 1 / bar2_score 11
TIMESTEP 13517 / EPSILON 0.04999999999999416 / bar1_score 2 / bar2_score 11
TIMESTEP 13646 / EPSILON 0.04999999999999416 / bar1_score 2 / bar2_score 12
TIMESTEP 13692 / EPSILON 0.04999999999999416 / bar1_score 2 / bar2_score 13
TIMESTEP 13738 / EPSILON 0.04999999999999416 / bar1_score 2 / bar2_score 14
TIMESTEP 13784 / EPSILON 0.04999999999999416 / bar1_score 2 / bar2_score 15
TIMESTEP 13830 / EPSILON 0.04999999999999416 / bar1_score 2 / bar2_score 16
TIMESTEP 13959 / EPSILON 0.04999999999999416 / bar1_score 3 / bar2_score 16
TIMESTEP 14088 / EPSILON 0.04999999999999416 / bar1_score 3 / bar2_score 17
TIMESTEP 14217 / EPSILON 0.04999999999999416 / bar1_score 4 / bar2_score 17
TIMESTEP 14346 / EPSILON 0.04999999999999416 / bar1_score 4 / bar2_score 18
TIMESTEP 1447

TIMESTEP 20639 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 6
TIMESTEP 20685 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 7
TIMESTEP 20731 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 8
TIMESTEP 20777 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 9
TIMESTEP 20823 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 10
TIMESTEP 20869 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 11
TIMESTEP 20915 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 12
TIMESTEP 20961 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 13
TIMESTEP 21007 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 14
TIMESTEP 21053 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 15
TIMESTEP 21099 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 16
TIMESTEP 21145 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 17
TIMESTEP 21191 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 18
TIMESTEP 21237 /

TIMESTEP 27330 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 9
TIMESTEP 27376 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 10
TIMESTEP 27422 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 11
TIMESTEP 27468 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 12
TIMESTEP 27514 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 13
TIMESTEP 27560 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 14
TIMESTEP 27690 / EPSILON 0.04999999999999416 / bar1_score 1 / bar2_score 14
TIMESTEP 28071 / EPSILON 0.04999999999999416 / bar1_score 2 / bar2_score 14
TIMESTEP 28200 / EPSILON 0.04999999999999416 / bar1_score 2 / bar2_score 15
TIMESTEP 28331 / EPSILON 0.04999999999999416 / bar1_score 3 / bar2_score 15
TIMESTEP 28712 / EPSILON 0.04999999999999416 / bar1_score 4 / bar2_score 15
TIMESTEP 28841 / EPSILON 0.04999999999999416 / bar1_score 4 / bar2_score 16
TIMESTEP 28887 / EPSILON 0.04999999999999416 / bar1_score 4 / bar2_score 17
TIMESTEP 2893

...

TIMESTEP 229447 / EPSILON 0.04999999999999416 / bar1_score 12 / bar2_score 9
TIMESTEP 229828 / EPSILON 0.04999999999999416 / bar1_score 13 / bar2_score 9
TIMESTEP 229957 / EPSILON 0.04999999999999416 / bar1_score 13 / bar2_score 10
TIMESTEP 230086 / EPSILON 0.04999999999999416 / bar1_score 14 / bar2_score 10
TIMESTEP 230469 / EPSILON 0.04999999999999416 / bar1_score 15 / bar2_score 10
TIMESTEP 230766 / EPSILON 0.04999999999999416 / bar1_score 15 / bar2_score 11
TIMESTEP 230812 / EPSILON 0.04999999999999416 / bar1_score 15 / bar2_score 12
TIMESTEP 230941 / EPSILON 0.04999999999999416 / bar1_score 16 / bar2_score 12
TIMESTEP 231322 / EPSILON 0.04999999999999416 / bar1_score 17 / bar2_score 12
TIMESTEP 231451 / EPSILON 0.04999999999999416 / bar1_score 17 / bar2_score 13
TIMESTEP 231580 / EPSILON 0.04999999999999416 / bar1_score 18 / bar2_score 13
TIMESTEP 231709 / EPSILON 0.04999999999999416 / bar1_score 18 / bar2_score 14
TIMESTEP 231838 / EPSILON 0.04999999999999416 / bar1_score 19 / ba

TIMESTEP 254465 / EPSILON 0.04999999999999416 / bar1_score 1 / bar2_score 0
TIMESTEP 254846 / EPSILON 0.04999999999999416 / bar1_score 2 / bar2_score 0
TIMESTEP 254975 / EPSILON 0.04999999999999416 / bar1_score 2 / bar2_score 1
TIMESTEP 255104 / EPSILON 0.04999999999999416 / bar1_score 3 / bar2_score 1
TIMESTEP 255233 / EPSILON 0.04999999999999416 / bar1_score 3 / bar2_score 2
TIMESTEP 255362 / EPSILON 0.04999999999999416 / bar1_score 4 / bar2_score 2
TIMESTEP 255743 / EPSILON 0.04999999999999416 / bar1_score 5 / bar2_score 2
TIMESTEP 256124 / EPSILON 0.04999999999999416 / bar1_score 6 / bar2_score 2
TIMESTEP 256505 / EPSILON 0.04999999999999416 / bar1_score 7 / bar2_score 2
TIMESTEP 256886 / EPSILON 0.04999999999999416 / bar1_score 8 / bar2_score 2
TIMESTEP 257269 / EPSILON 0.04999999999999416 / bar1_score 9 / bar2_score 2
TIMESTEP 257398 / EPSILON 0.04999999999999416 / bar1_score 9 / bar2_score 3
TIMESTEP 257527 / EPSILON 0.04999999999999416 / bar1_score 10 / bar2_score 3
TIMESTEP 25

TIMESTEP 283062 / EPSILON 0.04999999999999416 / bar1_score 17 / bar2_score 9
TIMESTEP 283443 / EPSILON 0.04999999999999416 / bar1_score 18 / bar2_score 9
TIMESTEP 283824 / EPSILON 0.04999999999999416 / bar1_score 19 / bar2_score 9
TIMESTEP 284121 / EPSILON 0.04999999999999416 / bar1_score 19 / bar2_score 10
TIMESTEP 284335 / EPSILON 0.04999999999999416 / bar1_score 19 / bar2_score 11
TIMESTEP 284464 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 0
TIMESTEP 284845 / EPSILON 0.04999999999999416 / bar1_score 1 / bar2_score 0
TIMESTEP 285226 / EPSILON 0.04999999999999416 / bar1_score 2 / bar2_score 0
TIMESTEP 285607 / EPSILON 0.04999999999999416 / bar1_score 3 / bar2_score 0
TIMESTEP 285904 / EPSILON 0.04999999999999416 / bar1_score 3 / bar2_score 1
TIMESTEP 285950 / EPSILON 0.04999999999999416 / bar1_score 3 / bar2_score 2
TIMESTEP 286079 / EPSILON 0.04999999999999416 / bar1_score 4 / bar2_score 2
TIMESTEP 286462 / EPSILON 0.04999999999999416 / bar1_score 5 / bar2_score 2
TIMES

TIMESTEP 311524 / EPSILON 0.04999999999999416 / bar1_score 8 / bar2_score 5
TIMESTEP 311653 / EPSILON 0.04999999999999416 / bar1_score 9 / bar2_score 5
TIMESTEP 312034 / EPSILON 0.04999999999999416 / bar1_score 10 / bar2_score 5
TIMESTEP 312417 / EPSILON 0.04999999999999416 / bar1_score 11 / bar2_score 5
TIMESTEP 312798 / EPSILON 0.04999999999999416 / bar1_score 12 / bar2_score 5
TIMESTEP 312927 / EPSILON 0.04999999999999416 / bar1_score 12 / bar2_score 6
TIMESTEP 313056 / EPSILON 0.04999999999999416 / bar1_score 13 / bar2_score 6
TIMESTEP 313438 / EPSILON 0.04999999999999416 / bar1_score 14 / bar2_score 6
TIMESTEP 313567 / EPSILON 0.04999999999999416 / bar1_score 14 / bar2_score 7
TIMESTEP 313696 / EPSILON 0.04999999999999416 / bar1_score 15 / bar2_score 7
TIMESTEP 314077 / EPSILON 0.04999999999999416 / bar1_score 16 / bar2_score 7
TIMESTEP 314458 / EPSILON 0.04999999999999416 / bar1_score 17 / bar2_score 7
TIMESTEP 314839 / EPSILON 0.04999999999999416 / bar1_score 18 / bar2_score 7
T

TIMESTEP 343542 / EPSILON 0.04999999999999416 / bar1_score 15 / bar2_score 0
TIMESTEP 343923 / EPSILON 0.04999999999999416 / bar1_score 16 / bar2_score 0
TIMESTEP 344304 / EPSILON 0.04999999999999416 / bar1_score 17 / bar2_score 0
TIMESTEP 344687 / EPSILON 0.04999999999999416 / bar1_score 18 / bar2_score 0
TIMESTEP 345068 / EPSILON 0.04999999999999416 / bar1_score 19 / bar2_score 0
Game_Ends_in Time: 38017
